# Modelo de Clasificación - Contenidos

## Objetivo
Predecir si un asegurado presentará siniestros en la cobertura de Contenidos (clasificación binaria).

## Pipeline
1. **Fase 1**: Preprocesamiento + GLM + LazyPredict
2. **Fase 2**: Optimización de top 5 modelos con GridSearchCV
3. **Fase 3**: Evaluación y selección del mejor modelo

**Dataset**: Por determinar | Desbalanceo: Por calcular

## 1. Imports y Configuración

In [46]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Preprocesamiento
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, precision_recall_curve, make_scorer
)

# LazyPredict
from lazypredict.Supervised import LazyClassifier

# Selección de variables
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga y Preparación de Datos

In [47]:
# Cargar datos
df = pd.read_csv("../data/processed/contenidos_full.csv")

print(f"Dataset original: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

Dataset original: (7999, 9)

Columnas: ['año_cursado', 'estudios_area', 'calif_promedio', '2_o_mas_inquilinos', 'distancia_al_campus', 'genero', 'extintor_incendios', 'Contenidos_siniestros_num', 'Contenidos_siniestros_monto']


,año_cursado,estudios_area,calif_promedio,2_o_mas_inquilinos,distancia_al_campus,genero,extintor_incendios,Contenidos_siniestros_num,Contenidos_siniestros_monto
0,3er año,Humanidades,3.01,No,0.00,Masculino,Si,0.00,0.00
1,3er año,Ciencias,1.52,No,0.00,Femenino,Si,0.00,0.00
2,3er año,Administracion,7.68,No,0.22,Femenino,No,0.00,0.00
3,4to año,Administracion,8.06,No,0.00,Masculino,No,0.00,0.00
4,2do año,Administracion,6.72,No,0.00,Femenino,No,0.00,0.00


In [48]:
# Eliminar columna de monto (no se usa en clasificación)
df = df.drop('Contenidos_siniestros_monto', axis=1)

# Crear variable binaria objetivo: 0 si no siniestró, 1 si siniestró
df['siniestrado'] = (df['Contenidos_siniestros_num'] > 0).astype(int)

# Eliminar columna original de número de siniestros
df = df.drop('Contenidos_siniestros_num', axis=1)

print(f"\nDataset preparado: {df.shape}")
print(f"\nDistribución variable objetivo:")
print(df['siniestrado'].value_counts())
print(f"\nPorcentaje siniestrados: {df['siniestrado'].mean()*100:.2f}%")
print(f"Desbalanceo: {df['siniestrado'].value_counts()[0] / df['siniestrado'].value_counts()[1]:.1f}:1")


Dataset preparado: (7999, 8)

Distribución variable objetivo:
siniestrado
0    7256
1     743
Name: count, dtype: int64

Porcentaje siniestrados: 9.29%
Desbalanceo: 9.8:1


## 3. Análisis Exploratorio Rápido

In [49]:
# Identificar tipos de variables
numeric_features = ['calif_promedio', 'distancia_al_campus']
categorical_features = ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']

print("Variables numéricas:", numeric_features)
print("Variables categóricas:", categorical_features)
print(f"\nTotal features: {len(numeric_features) + len(categorical_features)}")

# Verificar valores faltantes
print(f"\nValores faltantes:\n{df.isnull().sum()}")

Variables numéricas: ['calif_promedio', 'distancia_al_campus']
Variables categóricas: ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']

Total features: 7

Valores faltantes:
año_cursado            0
estudios_area          0
calif_promedio         0
2_o_mas_inquilinos     0
distancia_al_campus    0
genero                 0
extintor_incendios     0
siniestrado            0
dtype: int64


In [50]:
# Estadísticas descriptivas por clase
print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS POR CLASE")
print("=" * 80)

for var in categorical_features:
    print(f"\n{var}:")
    tabla = pd.crosstab(df[var], df['siniestrado'], normalize='index') * 100
    print(tabla.round(2))

ESTADÍSTICAS DESCRIPTIVAS POR CLASE

año_cursado:
siniestrado     0     1
año_cursado            
1er año     89.57 10.43
2do año     89.81 10.19
3er año     90.48  9.52
4to año     92.04  7.96
posgrado    92.70  7.30

estudios_area:
siniestrado        0     1
estudios_area             
Administracion 89.85 10.15
Ciencias       90.49  9.51
Humanidades    91.38  8.62
Otro           91.16  8.84

2_o_mas_inquilinos:
siniestrado            0     1
2_o_mas_inquilinos            
No                 92.39  7.61
Si                 83.78 16.22

genero:
siniestrado      0     1
genero                  
Femenino     91.03  8.97
Masculino    90.41  9.59
No respuesta 88.35 11.65
Otro         91.73  8.27

extintor_incendios:
siniestrado            0    1
extintor_incendios           
No                 91.17 8.83
Si                 90.52 9.48


## 4. Preprocesamiento y Split de Datos

In [51]:
# Separar features y target
X = df.drop('siniestrado', axis=1)
y = df['siniestrado']

# Split train/test estratificado 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print(f"Train set: {X_train.shape} | Siniestrados: {y_train.sum()} ({y_train.mean()*100:.2f}%)")
print(f"Test set:  {X_test.shape} | Siniestrados: {y_test.sum()} ({y_test.mean()*100:.2f}%)")

Train set: (6399, 7) | Siniestrados: 594 (9.28%)
Test set:  (1600, 7) | Siniestrados: 149 (9.31%)


In [52]:
# Pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='drop'
)

# Ajustar y transformar datos
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Obtener nombres de features después de one-hot encoding
feature_names_num = numeric_features
feature_names_cat = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_names = list(feature_names_num) + list(feature_names_cat)

# Convertir a DataFrame (mantener índices originales para alinear con y_train/y_test)
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print(f"\nFeatures después de preprocesamiento: {len(feature_names)}")
print(f"Shape X_train: {X_train_df.shape}")
print(f"Shape X_test: {X_test_df.shape}")


Features después de preprocesamiento: 14
Shape X_train: (6399, 14)
Shape X_test: (1600, 14)


## 5. Fase 1 - GLM con Selección Backward de Variables

In [53]:
# Modelo logístico completo para referencia
X_train_const = sm.add_constant(X_train_df)
logit_full = sm.Logit(y_train, X_train_const)
result_full = logit_full.fit(disp=0)

print("=" * 80)
print("MODELO LOGÍSTICO COMPLETO (todas las variables)")
print("=" * 80)
print(result_full.summary())

MODELO LOGÍSTICO COMPLETO (todas las variables)
                           Logit Regression Results                           
Dep. Variable:            siniestrado   No. Observations:                 6399
Model:                          Logit   Df Residuals:                     6384
Method:                           MLE   Df Model:                           14
Date:                Thu, 09 Oct 2025   Pseudo R-squ.:                 0.03057
Time:                        22:56:16   Log-Likelihood:                -1917.0
converged:                       True   LL-Null:                       -1977.5
Covariance Type:            nonrobust   LLR p-value:                 4.210e-19
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.3266      0.143    -16.247      0.000      -2.607      -2.046
calif_promedio               -0.0665  

In [54]:
# Selección Backward basada en p-valores
def backward_elimination(X, y, significance_level=0.05):
    """
    Realiza selección backward de variables basada en p-valores del GLM.
    """
    features = list(X.columns)
    
    while True:
        X_const = sm.add_constant(X[features])
        model = sm.Logit(y, X_const).fit(disp=0)
        p_values = model.pvalues.iloc[1:]
        max_p_value = p_values.max()
        
        if max_p_value <= significance_level:
            break
            
        feature_to_remove = p_values.idxmax()
        features.remove(feature_to_remove)
        print(f"Eliminando: {feature_to_remove} (p-valor: {max_p_value:.4f})")
    
    return features, model

print("=" * 80)
print("SELECCIÓN BACKWARD DE VARIABLES (α = 0.1)")
print("=" * 80)
print()

selected_features, logit_backward = backward_elimination(X_train_df, y_train, significance_level=0.1)

print()
print("=" * 80)
print(f"VARIABLES SELECCIONADAS: {len(selected_features)}")
print("=" * 80)
for feat in selected_features:
    print(f"  • {feat}")
print()
print(logit_backward.summary())

SELECCIÓN BACKWARD DE VARIABLES (α = 0.1)

Eliminando: genero_Otro (p-valor: 0.9135)
Eliminando: extintor_incendios_Si (p-valor: 0.8911)
Eliminando: año_cursado_2do año (p-valor: 0.7256)
Eliminando: estudios_area_Ciencias (p-valor: 0.6211)
Eliminando: genero_Masculino (p-valor: 0.3242)
Eliminando: genero_No respuesta (p-valor: 0.2815)
Eliminando: estudios_area_Otro (p-valor: 0.2702)
Eliminando: año_cursado_3er año (p-valor: 0.2436)
Eliminando: calif_promedio (p-valor: 0.1282)
Eliminando: estudios_area_Humanidades (p-valor: 0.1053)

VARIABLES SELECCIONADAS: 4
  • distancia_al_campus
  • año_cursado_4to año
  • año_cursado_posgrado
  • 2_o_mas_inquilinos_Si

                           Logit Regression Results                           
Dep. Variable:            siniestrado   No. Observations:                 6399
Model:                          Logit   Df Residuals:                     6394
Method:                           MLE   Df Model:                            4
Date:              

In [55]:
# Expandir variables categóricas completas
def expand_categorical_features(selected_features, all_feature_names, categorical_features):
    selected_cat_vars = set()
    for feat in selected_features:
        for cat_var in categorical_features:
            if feat.startswith(cat_var + '_'):
                selected_cat_vars.add(cat_var)
                break
    
    final_features = []
    for feat in selected_features:
        is_categorical = any(feat.startswith(cat_var + '_') for cat_var in categorical_features)
        if not is_categorical:
            final_features.append(feat)
    
    for feat in all_feature_names:
        for cat_var in selected_cat_vars:
            if feat.startswith(cat_var + '_') and feat not in final_features:
                final_features.append(feat)
                break
    
    return sorted(final_features)

final_selected_features = expand_categorical_features(selected_features, feature_names, categorical_features)

print("=" * 80)
print("FEATURES FINALES (con todos los niveles de categóricas)")
print("=" * 80)
print(f"\nTotal features: {len(final_selected_features)}")
print("\nFeatures seleccionadas:")
for feat in final_selected_features:
    indicator = "✓" if feat in selected_features else "+"
    print(f"  {indicator} {feat}")

X_train_selected = X_train_df[final_selected_features]
X_test_selected = X_test_df[final_selected_features]

print(f"\nShape final - Train: {X_train_selected.shape} | Test: {X_test_selected.shape}")

FEATURES FINALES (con todos los niveles de categóricas)

Total features: 6

Features seleccionadas:
  ✓ 2_o_mas_inquilinos_Si
  + año_cursado_2do año
  + año_cursado_3er año
  ✓ año_cursado_4to año
  ✓ año_cursado_posgrado
  ✓ distancia_al_campus

Shape final - Train: (6399, 6) | Test: (1600, 6)


## 6. Fase 1 - LazyPredict para Identificar Mejores Algoritmos

In [56]:
# Ejecutar LazyPredict
print("=" * 80)
print("EJECUTANDO LAZYPREDICT")
print("=" * 80)
print(f"\nDataset: {X_train_selected.shape[0]} muestras, {X_train_selected.shape[1]} features")
print("Esto puede tomar varios minutos...\n")

clf = LazyClassifier(verbose=0, ignore_warnings=True, predictions=True, random_state=42)
models, predictions = clf.fit(X_train_selected, X_test_selected, y_train, y_test)

print("\n" + "=" * 80)
print("RESULTADOS DE LAZYPREDICT")
print("=" * 80)
print()
print(models)

EJECUTANDO LAZYPREDICT

Dataset: 6399 muestras, 6 features
Esto puede tomar varios minutos...



  0%|          | 0/32 [00:00<?, ?it/s]


RESULTADOS DE LAZYPREDICT

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NearestCentroid                    0.73               0.55     0.55      0.78   
Perceptron                         0.79               0.52     0.52      0.81   
ExtraTreeClassifier                0.89               0.52     0.52      0.86   
DecisionTreeClassifier             0.88               0.51     0.51      0.86   
BaggingClassifier                  0.89               0.51     0.51      0.86   
RandomForestClassifier             0.88               0.51     0.51      0.86   
LGBMClassifier                     0.91               0.51     0.51      0.86   
ExtraTreesClassifier               0.88               0.51     0.51      0.85   
LabelPropagation                   0.91               0.50     0.50      0.86   
LabelSpreading                     0.91               0.50     0.50      0.86   


---
## Top 5 Modelos por F1-Score

In [57]:
# Top 5 modelos + modelos adicionales solicitados
print("=" * 80)
print("TOP 5 MODELOS POR F1-SCORE")
print("=" * 80)
print()

top_5_models = models.nlargest(5, 'F1 Score')
print(top_5_models)

# Agregar modelos adicionales solicitados explícitamente
modelos_adicionales = ['NearestCentroid', 'LogisticRegression', 'XGBClassifier', 'RandomForestClassifier']
modelos_para_optimizar = list(set(list(top_5_models.index) + modelos_adicionales))

print("\n" + "=" * 80)
print("MODELOS SELECCIONADOS PARA FASE 2 (GridSearchCV)")
print("=" * 80)
print(f"\nTotal modelos a optimizar: {len(modelos_para_optimizar)}")
print("\nDe LazyPredict (Top 5):")
for i, model_name in enumerate(top_5_models.index, 1):
    f1 = top_5_models.loc[model_name, 'F1 Score']
    acc = top_5_models.loc[model_name, 'Accuracy']
    print(f"{i}. {model_name} - F1: {f1:.4f} | Accuracy: {acc:.4f}")

print("\nModelos adicionales solicitados:")
for model_name in modelos_adicionales:
    if model_name in models.index:
        f1 = models.loc[model_name, 'F1 Score']
        acc = models.loc[model_name, 'Accuracy']
        print(f"  • {model_name} - F1: {f1:.4f} | Accuracy: {acc:.4f}")
    else:
        print(f"  • {model_name} - (no evaluado en LazyPredict)")

TOP 5 MODELOS POR F1-SCORE

                             Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                         
LGBMClassifier                   0.91               0.51     0.51      0.86   
LabelPropagation                 0.91               0.50     0.50      0.86   
LabelSpreading                   0.91               0.50     0.50      0.86   
PassiveAggressiveClassifier      0.91               0.50     0.50      0.86   
CalibratedClassifierCV           0.91               0.50     0.50      0.86   

                             Time Taken  
Model                                    
LGBMClassifier                     0.07  
LabelPropagation                   0.54  
LabelSpreading                     0.94  
PassiveAggressiveClassifier        0.01  
CalibratedClassifierCV             0.03  

MODELOS SELECCIONADOS PARA FASE 2 (GridSearchCV)

Total modelos a optimizar: 9

De LazyPredict (Top 5):
1. LGBMClassifier -

---
## FASE 2: Optimización de Hiperparámetros con GridSearchCV

In [58]:
# Funciones de evaluación
def gini_coefficient(y_true, y_pred_proba):
    auc = roc_auc_score(y_true, y_pred_proba)
    return 2 * auc - 1

def find_optimal_threshold(y_true, y_pred_proba, metric='f1'):
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        if metric == 'f1':
            score = f1_score(y_true, y_pred)
        elif metric == 'precision':
            score = precision_score(y_true, y_pred, zero_division=0)
        elif metric == 'recall':
            score = recall_score(y_true, y_pred)
        scores.append(score)
    
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx], scores[optimal_idx]

print("✓ Funciones de evaluación definidas")

✓ Funciones de evaluación definidas


In [59]:
# Importar modelos adicionales
from sklearn.linear_model import Perceptron
from sklearn.ensemble import ExtraTreesClassifier

print("✓ Modelos adicionales importados")

✓ Modelos adicionales importados


In [60]:
# Grillas de hiperparámetros con parámetros de suavizado/regularización (OPTIMIZADAS)
param_grids = {
    'LogisticRegression': {
        'C': [0.01, 0.1, 1.0, 10.0],  # Inverso de fuerza de regularización
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],  # Compatible con l1 y l2
        'class_weight': ['balanced', None],
        'max_iter': [1000]
    },
    
    'SVC': {
        'C': [0.1, 1.0, 10.0],  # Regularización
        'kernel': ['linear', 'rbf'],  # Solo los más comunes
        'gamma': ['scale', 'auto'],
        'class_weight': ['balanced', None],
        'probability': [True],  # Necesario para predict_proba
        'random_state': [42]
    },
    
    'NearestCentroid': {
        'metric': ['euclidean', 'manhattan'],
        'shrink_threshold': [None, 0.5, 1.0, 2.0]
    },
    
    'Perceptron': {
        'penalty': ['l2', 'l1', None],
        'alpha': [0.0001, 0.001, 0.01],
        'class_weight': ['balanced', None],
        'max_iter': [1000],
        'random_state': [42]
    },
    
    'LGBMClassifier': {
        'learning_rate': [0.01, 0.1],
        'num_leaves': [15, 31],
        'max_depth': [3, 5],  # Reducido de 3 a 2
        'n_estimators': [50, 100],
        'subsample': [0.8, 1.0],
        'reg_alpha': [0, 0.1],  # L1 regularization
        'reg_lambda': [1.0],  # L2 regularization (fijo)
        'class_weight': ['balanced', None],
        'random_state': [42],
        'verbose': [-1]
    },
    
    'ExtraTreesClassifier': {
        'n_estimators': [50, 100],
        'max_depth': [3, 5, None],  # Reducido de 4 a 3
        'min_samples_split': [10, 20],
        'min_samples_leaf': [5, 10],
        'max_features': ['sqrt'],  # Solo sqrt (reducido de 2 a 1)
        'class_weight': ['balanced', None],
        'ccp_alpha': [0.0, 0.01],
        'bootstrap': [True],  # Solo True (reducido de 2 a 1)
        'random_state': [42]
    },
    
    'RandomForestClassifier': {
        'n_estimators': [50, 100],
        'max_depth': [3, 5, None],  # Reducido de 4 a 3
        'min_samples_split': [10, 20],
        'min_samples_leaf': [5, 10],
        'max_features': ['sqrt'],  # Solo sqrt (reducido de 2 a 1)
        'class_weight': ['balanced', None],
        'ccp_alpha': [0.0, 0.01],
        'random_state': [42]
    }
}

model_mapping = {
    'LogisticRegression': LogisticRegression(random_state=42),
    'SVC': SVC(random_state=42),
    'NearestCentroid': NearestCentroid(),
    'Perceptron': Perceptron(random_state=42),
    'LGBMClassifier': LGBMClassifier(random_state=42, verbose=-1),
    'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42),
    'RandomForestClassifier': RandomForestClassifier(random_state=42)
}

modelos_para_optimizar = list(model_mapping.keys())

print("✓ Grillas OPTIMIZADAS con parámetros de regularización/suavizado:")
print("\n📊 Combinaciones por modelo:")
print("  - LogisticRegression: 4×2×2 = 16 combinaciones × 5 folds = 80 fits")
print("  - SVC: 3×2×2×2 = 24 combinaciones × 5 folds = 120 fits")
print("  - NearestCentroid: 2×4 = 8 combinaciones × 5 folds = 40 fits")
print("  - Perceptron: 3×3×2 = 18 combinaciones × 5 folds = 90 fits")
print("  - LGBMClassifier: 2×2×2×2×2×2×1×2 = 256 combinaciones × 5 folds = 1,280 fits ⚡")
print("  - ExtraTreesClassifier: 2×3×2×2×1×2×2×1 = 192 combinaciones × 5 folds = 960 fits ⚡")
print("  - RandomForestClassifier: 2×3×2×2×1×2×2 = 192 combinaciones × 5 folds = 960 fits ⚡")
print(f"\n📈 Total modelos a optimizar: {len(modelos_para_optimizar)}")
print("\n⚡ Grillas SIGNIFICATIVAMENTE reducidas para mejor rendimiento")

✓ Grillas OPTIMIZADAS con parámetros de regularización/suavizado:

📊 Combinaciones por modelo:
  - LogisticRegression: 4×2×2 = 16 combinaciones × 5 folds = 80 fits
  - SVC: 3×2×2×2 = 24 combinaciones × 5 folds = 120 fits
  - NearestCentroid: 2×4 = 8 combinaciones × 5 folds = 40 fits
  - Perceptron: 3×3×2 = 18 combinaciones × 5 folds = 90 fits
  - LGBMClassifier: 2×2×2×2×2×2×1×2 = 256 combinaciones × 5 folds = 1,280 fits ⚡
  - ExtraTreesClassifier: 2×3×2×2×1×2×2×1 = 192 combinaciones × 5 folds = 960 fits ⚡
  - RandomForestClassifier: 2×3×2×2×1×2×2 = 192 combinaciones × 5 folds = 960 fits ⚡

📈 Total modelos a optimizar: 7

⚡ Grillas SIGNIFICATIVAMENTE reducidas para mejor rendimiento


In [61]:
# GridSearchCV
print("=" * 80)
print("GRID SEARCH CON VALIDACIÓN CRUZADA")
print("=" * 80)
print()

f1_scorer = make_scorer(f1_score)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

optimized_models = {}
grid_results = {}

for i, model_name in enumerate(modelos_para_optimizar, 1):
    print(f"\n{'='*80}")
    print(f"[{i}/{len(modelos_para_optimizar)}] Optimizando: {model_name}")
    print(f"{'='*80}")
    
    if model_name not in param_grids or model_name not in model_mapping:
        print(f"⚠ No hay grilla para {model_name}, saltando...")
        continue
    
    base_model = model_mapping[model_name]
    param_grid = param_grids[model_name]
    
    n_combinations = np.prod([len(v) if isinstance(v, list) else 1 for v in param_grid.values()])
    print(f"Combinaciones: {int(n_combinations)} | Total fits: {int(5 * n_combinations)}")
    
    try:
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            cv=cv,
            scoring=f1_scorer,
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train_selected, y_train)
        
        optimized_models[model_name] = grid_search.best_estimator_
        grid_results[model_name] = {
            'best_params': grid_search.best_params_,
            'best_score_cv': grid_search.best_score_
        }
        
        print(f"✓ F1-Score CV: {grid_search.best_score_:.4f}")
        print(f"Mejores parámetros: {grid_search.best_params_}")
        
    except Exception as e:
        print(f"✗ Error: {str(e)}")

print(f"\n{'='*80}")
print(f"Completado: {len(optimized_models)}/{len(modelos_para_optimizar)} modelos")

GRID SEARCH CON VALIDACIÓN CRUZADA


[1/7] Optimizando: LogisticRegression
Combinaciones: 16 | Total fits: 80
✓ F1-Score CV: 0.2198
Mejores parámetros: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}

[2/7] Optimizando: SVC
Combinaciones: 24 | Total fits: 120
✓ F1-Score CV: 0.2278
Mejores parámetros: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear', 'probability': True, 'random_state': 42}

[3/7] Optimizando: NearestCentroid
Combinaciones: 8 | Total fits: 40
✓ F1-Score CV: 0.2278
Mejores parámetros: {'metric': 'euclidean', 'shrink_threshold': 2.0}

[4/7] Optimizando: Perceptron
Combinaciones: 18 | Total fits: 90
✓ F1-Score CV: 0.1617
Mejores parámetros: {'alpha': 0.0001, 'class_weight': 'balanced', 'max_iter': 1000, 'penalty': 'l2', 'random_state': 42}

[5/7] Optimizando: LGBMClassifier
Combinaciones: 128 | Total fits: 640
✓ F1-Score CV: 0.2171
Mejores parámetros: {'class_weight': 'balanced', 'learning_rate': 0

In [62]:
# Evaluación en test set
print("=" * 80)
print("EVALUACIÓN EN TEST SET")
print("=" * 80)

test_results = []

for model_name, model in optimized_models.items():
    print(f"\n{'='*80}")
    print(f"Evaluando: {model_name}")
    print(f"{'='*80}")
    
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
    elif hasattr(model, 'decision_function'):
        y_pred_proba = model.decision_function(X_test_selected)
        y_pred_proba = (y_pred_proba - y_pred_proba.min()) / (y_pred_proba.max() - y_pred_proba.min())
    else:
        y_pred_proba = model.predict(X_test_selected).astype(float)
    
    optimal_threshold, _ = find_optimal_threshold(y_test, y_pred_proba)
    y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred_optimal)
    precision = precision_score(y_test, y_pred_optimal, zero_division=0)
    recall = recall_score(y_test, y_pred_optimal, zero_division=0)
    f1 = f1_score(y_test, y_pred_optimal, zero_division=0)
    
    try:
        auc = roc_auc_score(y_test, y_pred_proba)
        gini = gini_coefficient(y_test, y_pred_proba)
    except:
        auc = gini = np.nan
    
    test_results.append({
        'Modelo': model_name,
        'F1_CV': grid_results[model_name]['best_score_cv'],
        'F1_Test': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'AUC': auc,
        'Gini': gini,
        'Threshold': optimal_threshold
    })
    
    print(f"Threshold: {optimal_threshold:.3f}")
    print(f"F1 (CV): {grid_results[model_name]['best_score_cv']:.4f} | F1 (Test): {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
    print(f"AUC: {auc:.4f} | Gini: {gini:.4f}")
    
    cm = confusion_matrix(y_test, y_pred_optimal)
    print(f"\nMatriz de Confusión:")
    print(f"  TN: {cm[0,0]:4d} | FP: {cm[0,1]:4d}")
    print(f"  FN: {cm[1,0]:4d} | TP: {cm[1,1]:4d}")

df_results = pd.DataFrame(test_results).sort_values('F1_Test', ascending=False)

print(f"\n{'='*80}")
print("RESUMEN DE RESULTADOS")
print(f"{'='*80}")
print()
print(df_results.to_string(index=False))

EVALUACIÓN EN TEST SET

Evaluando: LogisticRegression
Threshold: 0.580
F1 (CV): 0.2198 | F1 (Test): 0.1995
Accuracy: 0.7844 | Precision: 0.1525 | Recall: 0.2886
AUC: 0.5416 | Gini: 0.0832

Matriz de Confusión:
  TN: 1212 | FP:  239
  FN:  106 | TP:   43

Evaluando: SVC
Threshold: 0.100
F1 (CV): 0.2278 | F1 (Test): 0.1882
Accuracy: 0.7681 | Precision: 0.1396 | Recall: 0.2886
AUC: 0.5028 | Gini: 0.0055

Matriz de Confusión:
  TN: 1186 | FP:  265
  FN:  106 | TP:   43

Evaluando: NearestCentroid
Threshold: 0.430
F1 (CV): 0.2278 | F1 (Test): 0.1882
Accuracy: 0.7681 | Precision: 0.1396 | Recall: 0.2886
AUC: 0.5336 | Gini: 0.0672

Matriz de Confusión:
  TN: 1186 | FP:  265
  FN:  106 | TP:   43

Evaluando: Perceptron
Threshold: 0.740
F1 (CV): 0.1617 | F1 (Test): 0.1786
Accuracy: 0.4713 | Precision: 0.1044 | Recall: 0.6174
AUC: 0.5221 | Gini: 0.0443

Matriz de Confusión:
  TN:  662 | FP:  789
  FN:   57 | TP:   92

Evaluando: LGBMClassifier
Threshold: 0.500
F1 (CV): 0.2171 | F1 (Test): 0.1977

---
## Exportación LGBMClassifier con Dataset Completo

In [63]:
# Reentrenar LGBMClassifier con TODO el dataset usando mejores hiperparámetros
print("=" * 80)
print("REENTRENAMIENTO LGBM CON DATASET COMPLETO")
print("=" * 80)
print()

# Combinar train y test
X_full = pd.concat([X_train_selected, X_test_selected])
y_full = pd.concat([y_train, y_test])

print(f"Dataset completo: {X_full.shape}")
print(f"Siniestrados totales: {y_full.sum()} ({y_full.mean()*100:.2f}%)")

# Obtener los mejores parámetros del LGBMClassifier
best_params_lgbm = grid_results['LGBMClassifier']['best_params']
print(f"\nMejores parámetros del LGBMClassifier:")
for param, value in best_params_lgbm.items():
    print(f"  {param}: {value}")

# Crear y entrenar modelo con todos los datos
lgbm_final = LGBMClassifier(**best_params_lgbm)
lgbm_final.fit(X_full, y_full)

print(f"\n✓ LGBMClassifier reentrenado con {len(X_full)} registros")
print(f"✓ Listo para exportar")

REENTRENAMIENTO LGBM CON DATASET COMPLETO

Dataset completo: (7999, 6)
Siniestrados totales: 743 (9.29%)

Mejores parámetros del LGBMClassifier:
  class_weight: balanced
  learning_rate: 0.01
  max_depth: 3
  n_estimators: 50
  num_leaves: 15
  random_state: 42
  reg_alpha: 0
  reg_lambda: 1.0
  subsample: 0.8
  verbose: -1

✓ LGBMClassifier reentrenado con 7999 registros
✓ Listo para exportar


In [64]:
import pickle
import os

# Crear directorio models si no existe
os.makedirs('../models', exist_ok=True)

# Preparar objeto para exportar
modelo_export = {
    'modelo': lgbm_final,
    'preprocessor': preprocessor,
    'features_seleccionadas': final_selected_features,
    'threshold_optimo': df_results[df_results['Modelo'] == 'LGBMClassifier']['Threshold'].values[0],
    'metricas': {
        'F1_CV': grid_results['LGBMClassifier']['best_score_cv'],
        'F1_Test': df_results[df_results['Modelo'] == 'LGBMClassifier']['F1_Test'].values[0],
        'Accuracy': df_results[df_results['Modelo'] == 'LGBMClassifier']['Accuracy'].values[0],
        'Precision': df_results[df_results['Modelo'] == 'LGBMClassifier']['Precision'].values[0],
        'Recall': df_results[df_results['Modelo'] == 'LGBMClassifier']['Recall'].values[0],
        'AUC': df_results[df_results['Modelo'] == 'LGBMClassifier']['AUC'].values[0],
        'Gini': df_results[df_results['Modelo'] == 'LGBMClassifier']['Gini'].values[0]
    },
    'mejores_parametros': grid_results['LGBMClassifier']['best_params']
}

# Exportar modelo
model_path = '../models/clasificacion_contenidos.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(modelo_export, f)

print("=" * 80)
print("MODELO LGBM EXPORTADO EXITOSAMENTE")
print("=" * 80)
print(f"\nRuta: {model_path}")
print(f"\nModelo: LGBMClassifier")
print(f"Mejores parámetros: {modelo_export['mejores_parametros']}")
print(f"\nMétricas en Test Set:")
print(f"  F1-Score:  {modelo_export['metricas']['F1_Test']:.4f}")
print(f"  Accuracy:  {modelo_export['metricas']['Accuracy']:.4f}")
print(f"  Precision: {modelo_export['metricas']['Precision']:.4f}")
print(f"  Recall:    {modelo_export['metricas']['Recall']:.4f}")
print(f"  AUC-ROC:   {modelo_export['metricas']['AUC']:.4f}")
print(f"  Gini:      {modelo_export['metricas']['Gini']:.4f}")
print(f"\nThreshold óptimo: {modelo_export['threshold_optimo']:.3f}")
print(f"\nContenido del archivo:")
print(f"  • modelo: LGBMClassifier entrenado con {len(X_full)} registros")
print(f"  • preprocessor: ColumnTransformer (StandardScaler + OneHotEncoder)")
print(f"  • features_seleccionadas: {final_selected_features}")
print(f"  • threshold_optimo: {modelo_export['threshold_optimo']:.3f}")
print(f"  • metricas: diccionario con todas las métricas")
print(f"  • mejores_parametros: {modelo_export['mejores_parametros']}")

MODELO LGBM EXPORTADO EXITOSAMENTE

Ruta: ../models/clasificacion_contenidos.pkl

Modelo: LGBMClassifier
Mejores parámetros: {'class_weight': 'balanced', 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'num_leaves': 15, 'random_state': 42, 'reg_alpha': 0, 'reg_lambda': 1.0, 'subsample': 0.8, 'verbose': -1}

Métricas en Test Set:
  F1-Score:  0.1977
  Accuracy:  0.7819
  Precision: 0.1503
  Recall:    0.2886
  AUC-ROC:   0.5484
  Gini:      0.0968

Threshold óptimo: 0.500

Contenido del archivo:
  • modelo: LGBMClassifier entrenado con 7999 registros
  • preprocessor: ColumnTransformer (StandardScaler + OneHotEncoder)
  • features_seleccionadas: ['2_o_mas_inquilinos_Si', 'año_cursado_2do año', 'año_cursado_3er año', 'año_cursado_4to año', 'año_cursado_posgrado', 'distancia_al_campus']
  • threshold_optimo: 0.500
  • metricas: diccionario con todas las métricas
  • mejores_parametros: {'class_weight': 'balanced', 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'num_